## Wiktionary extractor

In [132]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# response = requests.get("https://en.wiktionary.org/wiki/cinnamon")
# soup = BeautifulSoup(response.text, 'html.parser')
# soup

In [133]:
# Variables
key = 'cinnamon'
path = 'data\\wiktionary\\'

In [134]:
# Define the URL
url = f"https://en.wiktionary.org/wiki/{key}"

# Get the HTML content of the page
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Get the translations
ids = ['Translations-spice', 'Translations-\'\'Cinnamomum_verum\'\''] ### 

# Create a list of tuples containing the sense, language, and translation
translations = []
for id in ids:
    translations_div = soup.find('div', {'id': id})
    if translations_div is not None:
        for li in translations_div.find_all('li'):
            lang_and_trans = li.get_text().split(':')
            if len(lang_and_trans) == 2:
                lang = lang_and_trans[0].strip()
                trans = lang_and_trans[1].strip()
                translations.append((id, lang, trans))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(translations, columns=['sense', 'language', 'term'])

# Use regex to replace cells in the sense column of df
df['sense'] = [re.sub("Translations-", "", str(x)) for x in df['sense']]

# # Split the rows with multiple translations into separate rows
# df['term'] = df['term'].str.split(', ')
# df = df.explode('term')

# Define a function to split on commas not inside parentheses
def split_not_in_parentheses(s):
    return re.split(r',\s*(?![^()]*\))', s)

# Apply the function to the 'term' column
df['term'] = df['term'].apply(split_not_in_parentheses)

# Explode the 'term' column
df = df.explode('term')

# Save the DataFrame to a CSV file
# df.to_csv(path + 'raw_translations_for_check.csv', index=False)

# Drop duplicates
df = df.drop_duplicates(subset = ['language', 'term'], keep = 'first').reset_index(drop = True)

# Get list of senses
list_of_senses = df['sense'].unique().tolist()

# Filter duplicates among senses and get rid of B name if A is found. # Change subset to be more strict e.g. subset=['language', 'term']
if len(list_of_senses) > 1:
    df['duplicate'] = df[df.duplicated(subset=['language', 'term'], keep=False)]['sense'] == list_of_senses[1]
    df.drop(df[df['duplicate'] == True].index, inplace=True)
    df.drop(['duplicate'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)

# Cleaning
df['term'] = [re.sub('\xa0', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(bcl\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(nds\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(scn\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(ast\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(Föhr-Amrum\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s?\(\w\w\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(please verify\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s+', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' ,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r'^\s', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s$', "", str(x)) for x in df['term']]

# Other
df['term'] = [re.sub(r"\(taraškievica\)", "", str(x)) for x in df['term']]
df['term'] = [re.sub(r"class 9/10", "", str(x)) for x in df['term']]

# Change ( and ) to * and * 
df['term'] = [re.sub(r'\(', "*", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\)', "*", str(x)) for x in df['term']]

# Remove ⁧ and ⁩ from term
df['term'] = [re.sub(r'⁧', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'⁩', "", str(x)) for x in df['term']]

# drop NA
df = df[df.term != "please add this translation if you can"]

# If there is a star in the term column, split the column into 'script' and 'transliteration' columns
df['word'] = df['term'].apply(lambda x: x.split('*')[0].strip())
df['transliteration'] = df['term'].apply(lambda x: x.split('*')[1].strip() if len(x.split('*')) > 1 else None)

# Reorder by alphabetizing the language column
df = df.sort_values('language').reset_index(drop=True)

# Create source
df['source'] = 'Wiktionary'
df['group'] = ''
df['skip'] = ''

# reorder
df = df[['skip', 'sense', 'language', 'term', 'group', 'word', 'transliteration', 'source']]

# Save the DataFrame to a CSV file
df.to_csv(path + f'{key}_gen.csv', index=False)
df.to_excel(path + f'{key}_gen.xlsx', sheet_name='wiktionary', index=None)

# Print
df.head(60)

,skip,sense,language,term,group,word,transliteration,source
0,,spice,Afrikaans,kaneel,,kaneel,None,Wiktionary
1,,''Cinnamomum_verum'',Afrikaans,kaneelboom,,kaneelboom,None,Wiktionary
2,,spice,Albanian,kanellë,,kanellë,None,Wiktionary
3,,spice,Amharic,ቀረፋ *ḳäräfa*,,ቀረፋ,ḳäräfa,Wiktionary
4,,''Cinnamomum_verum'',Armenian,դարչին *darčʿin*,,դարչին,darčʿin,Wiktionary
5,,''Cinnamomum_verum'',Assamese,দালচেনি *dalseni*,,দালচেনি,dalseni,Wiktionary
6,,spice,Azerbaijani,darçın,,darçın,None,Wiktionary
7,,''Cinnamomum_verum'',Basque,kanela,,kanela,None,Wiktionary
8,,spice,Belarusian,кары́ца *karýca*,,кары́ца,karýca,Wiktionary
9,,spice,Belarusian,цынамо́н *cynamón*,,цынамо́н,cynamón,Wiktionary


# End

In [129]:
# Change languages to Glottolog name
def glottologize():
    df['language'] = [re.sub(r"^Arabic$", "Standard Arabic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Armenian$", "Eastern Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Armenian (Eastern)$", "Eastern Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Armenian (Western)$", "Western Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Bikol Central$", "Coastal-Naga Bikol", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Bosnian$", "Bosnian Standard", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Croatian$", "Croatian Standard", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Gaelic$", "Scottish Gaelic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Greek$", "Modern Greek", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Greenlandic$", "Kalaallisut", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Hebrew$", "Modern Hebrew", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Kyrgyz$", "Kirghiz", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Luxembourgish$", "Luxemburgish", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Mandarin$", "Mandarin Chinese", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Manipuri (Meitei-Lon)$", "Manipuri", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Sumi)$", "Sumi Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Tangkhul)$", "North-Central Tangkhul Naga", str(x)) for x in df['language']] # A hypoglot
    df['language'] = [re.sub(r"^Naga (Rengma)$", "Northern Rengma Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Lotha)$", "Lotha Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Konyak)$", "Konyak Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Chakhesang-Chokri)$", "Chokri Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Ao)$", "Ao Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Naga (Angami)$", "Angami Naga", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Norman$", "Anglo-Norman", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^North Frisian$", "Northern Frisian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^North Sami$", "North Saami", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old Armenian$", "Classical-Middle Armenian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old Church Slavonic$", "Church Slavic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old East Slavic$", "Old Russian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Old Javanese$", "Kawi", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Ossetian$", "Modern Ossetic", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Punjabi$", "Eastern Panjabi", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Serbian$", "Serbian Standard", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^Serbo-Croatian$", "Serbian-Croatian-Bosnian", str(x)) for x in df['language']]
    df['language'] = [re.sub(r"^West Frisian$", "Western Frisian", str(x)) for x in df['language']]

    return df

glottologize()

,sense,language,term,group,word,transliteration,source
0,spice,Afrikaans,kaneel,,kaneel,None,Wiktionary
1,''Cinnamomum_verum'',Afrikaans,kaneelboom,,kaneelboom,None,Wiktionary
2,spice,Albanian,kanellë,,kanellë,None,Wiktionary
3,spice,Amharic,ቀረፋ *ḳäräfa*,,ቀረፋ,ḳäräfa,Wiktionary
4,''Cinnamomum_verum'',Eastern Armenian,դարչին *darčʿin*,,դարչին,darčʿin,Wiktionary
...,...,...,...,...,...,...,...
157,spice,Volapük,kirfat,,kirfat,None,Wiktionary
158,''Cinnamomum_verum'',Volapük,kirfatep,,kirfatep,None,Wiktionary
159,spice,Welsh,synamon,,synamon,None,Wiktionary
160,spice,Yiddish,צימערינג *tsimering*,,צימערינג,tsimering,Wiktionary


## Manual steps

Now the manual work: Use the generated file `key_generated.xlsx` to fix, amend, append, group, and organize the names, with the addition of other sources and create a `key.xlsx` master list.

Steps:
 1. Manual check transliterations, especially Dhivehi.
 2. Analyze and group words/names.
 3. Mark uncertain ones for skipping with 'yes'.

Recommended sources:
* Katzer (needs serious checking)
* WOLD
* Max Planck databases (CLIC3, etc.)
* others... 

After that, more preprocessing, cleaning, and merging with language data and coordinates.

In [130]:
# Read in dataframe from xlsx
df = pd.read_excel(path + f'{key}_gen.xlsx', sheet_name='wiktionary')
df

,sense,language,term,group,word,transliteration,source
0,spice,Afrikaans,kaneel,NaN,kaneel,NaN,Wiktionary
1,''Cinnamomum_verum'',Afrikaans,kaneelboom,NaN,kaneelboom,NaN,Wiktionary
2,spice,Albanian,kanellë,NaN,kanellë,NaN,Wiktionary
3,spice,Amharic,ቀረፋ *ḳäräfa*,NaN,ቀረፋ,ḳäräfa,Wiktionary
4,''Cinnamomum_verum'',Armenian,դարչին *darčʿin*,NaN,դարչին,darčʿin,Wiktionary
...,...,...,...,...,...,...,...
157,spice,Volapük,kirfat,NaN,kirfat,NaN,Wiktionary
158,''Cinnamomum_verum'',Volapük,kirfatep,NaN,kirfatep,NaN,Wiktionary
159,spice,Welsh,synamon,NaN,synamon,NaN,Wiktionary
160,spice,Yiddish,צימערינג *tsimering*,NaN,צימערינג,tsimering,Wiktionary


In [131]:
# Extract only desired columns
selectlist = ['skip', 'language', 'term', 'transliteration', 'item', 'group']
df = df[selectlist]

print("Before skipping: ", df.shape[0])
df=df[df['skip'] != "yes"]
print("After skipping: ", df.shape[0])

# # drop columns manually
# df.drop(columns=['skip', 'literal', 'explanation', 'IPA', 'source zotero', 'notes', 'type', 'katzer', 'katzer tr', 'checked', 'reference', 'link'], inplace=True)

KeyError: "['skip', 'item'] not in index"

# FIX BELOW HERE

In [ ]:
# df_input = df.copy()

# # load in datasets
# languages=pd.read_csv(path_in_wiktionary + 'languages/languages.csv', header =[0], delimiter=',', encoding="utf-8", index_col=[0])

# # merge input and languages
# df = pd.merge(df_input, languages, on=['language'])
# print("Merged:", df.shape)

# #drop duplicates
# df.drop_duplicates(subset=['language', 'term'], keep='first', inplace=True, ignore_index=True)
# print("Dropping duplicates:", df.shape)

# multilingual = df
# multilingual

# #check missing ones
# temp = pd.merge(df_input, multilingual, how='outer', suffixes=('','_y'), indicator=True)
# missing = temp[temp['_merge']=='left_only'][df_input.columns]
# print("The following terms and languages have failed to load:")
# print(missing)

# # df = df.dropna() #OPERATIVE ONLY
# # df = df.fillna('x')

# #sort by categories, cinnamon ######## AUTOMATE ########
# df['group'] = pd.Categorical(df['group'], ["canela", "kinnamon", "korica", "qirfa", "darchin", "gui", "other"]) # add categorical order here
# df.sort_values("group", inplace = True) # sort according to the categories

# # #sort by categories, pepper ######## AUTOMATE ########
# # df['group'] = pd.Categorical(df['group'], ["pippali", "pigment", "marica", "hujiao", "other"]) # add categorical order here
# # df.sort_values("group", inplace = True) # sort according to the categories

# # create text for annotation label
# df['text'] = df['term'] + '<br>' + df['transliteration'].astype(str) + '<br>Language: ' + df['language'] + '<br>Family: ' + df['family']
# df['text'] = [re.sub(r"<br>nan<br>", "<br>", str(x)) for x in df['text']]

# df['term'] = [re.sub(r"\u200e", "", str(x)) for x in df['term']] #removes right to left mark
# df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!

# # reindex?

# # save
# df.to_csv(path_in_wiktionary + "multilingual/" + key +'.csv')
# df

### Multilingual extractions for Spices

In [ ]:
key = "cinnamon"

In [ ]:
# Open txt and separate every line
txt = open(path_in_wiktionary + key + '_spice.txt', "r", encoding = 'utf8')
lines = txt.readlines()
txt.close()

# Remove /n at the end of each line
for index, line in enumerate(lines):
      lines[index] = line.strip()

# Creating a dataframe
df = pd.DataFrame(columns=('first', 'second'))
i = 0  
first = "" 
second = ""  
for line in lines:
        #you have to kind of define what are the values in columns,for example second column includes:
        second = re.sub(r'', "", line)
        #this is how you create next line data
        df.loc[i] = [first, second]
        i =i+1


df['second'].replace('', np.nan, inplace=True)
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(['first'], axis=1, inplace=True)
df=df[df.second.str.contains(":", regex=True)]
df = pd.DataFrame(df.second.str.split(':', 1).tolist(), columns = ['language','term'])

df['sense'] = 'spice'

#fill empty cells backward (i.e fill the parent language content with value from a variant if the former is empty)
df.term[df['term']==""] = np.NaN
df['term'] =  df.term.str.extract('(.*)').fillna(method='bfill')

# df = df.sort_values('language')

spice = df
print(spice)

FileNotFoundError: [Errno 2] No such file or directory: 'data/wiktionary/cinnamon_spice.txt'

In [ ]:
path = path_in_wiktionary + key + '_plant.txt'

if os.path.exists(path):

      #open txt and seperate every line
      df = open(path_in_wiktionary + key + '_plant.txt', "r", encoding = 'utf8')
      lines = df.readlines()
      df.close()

      # remove /n at the end of each line
      for index, line in enumerate(lines):
            lines[index] = line.strip()

      #creating a dataframe(consider u want to convert your data to 2 columns)
      df = pd.DataFrame(columns=('first', 'second'))
      i = 0 
      first = "" 
      second = ""  
      for line in lines:
            #you have to kind of define what are the values in columns,for example second column includes:
            second = re.sub(r'', "", line)
            #this is how you create next line data
            df.loc[i] = [first, second]
            i =i+1

      df['second'].replace('', np.nan, inplace=True)
      df.dropna(axis=0, inplace=True)
      df.reset_index(drop=True, inplace=True)
      df.drop(['first'], axis=1, inplace=True)
      df=df[df.second.str.contains(":", regex=True)]
      df = pd.DataFrame(df.second.str.split(':', 1).tolist(), columns = ['language','term'])

      df['sense'] = 'plant'

      #fill empty cells backward (i.e fill the parent language content with value from a variant)
      df.term[df['term']==""] = np.NaN
      df['term'] =  df.term.str.extract('(.*)').fillna(method='bfill')
      # df = df.sort_values('language')

      plant = df
      print(plant)

           language                                   term  sense
0         Afrikaans                             kaneelboom  plant
1            Arabic                     قِرْفَة‎ f (qirfa)  plant
2     Hijazi Arabic                          قرفة‎ (girfa)  plant
3   Moroccan Arabic                          قرفة‎ (qarfa)  plant
4           Aramaic   ܕܪܨܝܢܝ‎ (dārṣīnī), ܨܝܢܕܪܓ‎ (ṣīndreḡ)  plant
..              ...                                    ...    ...
61             Thai                       อบเชย (òp-chəəi)  plant
62          Tibetan                     ཤིང་ཚ (shing tsha)  plant
63          Turkish                           tarçın ağacı  plant
64       Vietnamese                                cây quế  plant
65          Volapük                   kirfatep, kirfatabim  plant

[66 rows x 3 columns]


In [ ]:
# concat
frames = [spice, plant]
df = pd.concat(frames)

# sort by categories
df['sense'] = pd.Categorical(df['sense'], ["spice", "plant"]) # add categorical order here
df.sort_values("sense", inplace = True) # sort according to the categories

# sort
df = df.sort_values('language')

# drop duplicates
df = df.drop_duplicates(subset = ['language', 'term'], keep = 'first').reset_index(drop = True)

# reset index
df.reset_index(inplace=True, drop=True)

# filter duplicates and get rid of plant name if spice name found
df['duplicate'] = df[df.duplicated(subset=['language'], keep=False)]['sense']=="plant" # change subset to be more strict e.g. subset=['language', 'term']
df.drop(df[df['duplicate'] == True].index, inplace=True)
df.drop(['duplicate'], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,language,term,sense
0,Afrikaans,kaneel,spice
1,Albanian,kanellë (sq) f,spice
2,Amharic,ቀረፋ (ḳäräfa),spice
3,Ancient,κιννάμωμον n (kinnámōmon),spice
4,Arabic,قِرْفَة‎ f (qirfa),spice
...,...,...,...
104,Uzbek,"dolchin (uz), koritsa",spice
105,Vietnamese,quế (vi),spice
106,Volapük,kirfat,spice
107,Welsh,synamon m,spice


In [ ]:
# cleaning
df['term'] = [re.sub(r' m ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl ', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r' m$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' f$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' n$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' c$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r' pl$', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(bcl\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(nds\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(scn\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(ast\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(Föhr-Amrum\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s?\(\w\w\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\(please verify\)', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s+', " ", str(x)) for x in df['term']]
df['term'] = [re.sub(r' ,', ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r'^\s', "", str(x)) for x in df['term']]
df['term'] = [re.sub(r'\s$', "", str(x)) for x in df['term']]

# transliteration, formatting
df['transliteration'] = df.term.str.findall("(\((\w*\,?\.?\-?\:?\d?\s?)+\))").fillna(method='ffill')
df['transliteration'] = [re.sub(r"[\(\)\[\]]", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r", ''", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"^'", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"'$", "", str(x)) for x in df['transliteration']]
df['transliteration'] = [re.sub(r"', '", ", ", str(x)) for x in df['transliteration']]

# clean term of transliteration
df['term'] = [re.sub(r"(\((\w*\,?\.?\-?\:?\d?\s?)+\))", "", str(x)) for x in df['term']] 
df['term'] = [re.sub(r" +,", ",", str(x)) for x in df['term']]
df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!


# other
df['transliteration'] = [re.sub(r", taraškievica", "", str(x)) for x in df['transliteration']]

# drop NA
df = df[df.term != "please add this translation if you can"]

# create 'item' column where everythin is in
df['item'] = df['transliteration']
df['item'] = df['item'].replace('', pd.NA).fillna(df['term'])
df['source zotero'] = 'Wiktionary'

# reorder
df = df[['language', 'term', 'transliteration', 'item', 'sense']]

#write
df.to_excel(path_in_wiktionary + key + '_generated.xlsx', sheet_name='wiktionary', index=None, encoding="utf-8")
df

,language,term,transliteration,item,sense
0,Afrikaans,kaneel,,kaneel,spice
1,Albanian,kanellë,,kanellë,spice
2,Amharic,ቀረፋ,ḳäräfa,ḳäräfa,spice
3,Ancient,κιννάμωμον,kinnámōmon,kinnámōmon,spice
4,Standard Arabic,قِرْفَة‎,qirfa,qirfa,spice
...,...,...,...,...,...
104,Uzbek,"dolchin, koritsa",,"dolchin, koritsa",spice
105,Vietnamese,quế,,quế,spice
106,Volapük,kirfat,,kirfat,spice
107,Welsh,synamon,,synamon,spice


Now the manual work: Use `spice_out.xlsx` to fix, amend, append, group, and organize the names, with the addition of other sources and create a `spice.xlsx` master list.

Recommended sources:
* Katzer (needs serious checking)
* WOLD
* Max Planck databases (CLIC3, etc.)
* others... 

After that, more preprocessing, cleaning, and merging with language data and coordinates.

In [ ]:
# Read and store content of an excel file 
read_file = pd.read_excel(path_in_wiktionary + key +".xlsx")

# Write the dataframe object into csv file
read_file.to_csv (path_in_wiktionary + key + ".csv", index = None, header=True)

# Load in dataset
df=pd.read_csv(path_in_wiktionary + key + ".csv", header =[0], delimiter=',', encoding="utf-8")

# extract only desired columns
selectlist = ['skip', 'language', 'term', 'transliteration', 'item', 'group']
df = df[selectlist]

print("Before skipping: ", df.shape)
df=df[df['skip'] != "yes"]
print("After skipping: ", df.shape)

# # drop columns manually
# df.drop(columns=['skip', 'literal', 'explanation', 'IPA', 'source zotero', 'notes', 'type', 'katzer', 'katzer tr', 'checked', 'reference', 'link'], inplace=True)

# Change languages to glottolog name
df['language'] = [re.sub(r"^Arabic$", "Standard Arabic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Mandarin$", "Mandarin Chinese", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Armenian$", "Classical-Middle Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Croatian$", "Croatian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Serbian$", "Serbian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Bosnian$", "Bosnian Standard", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Serbo-Croatian$", "Serbian-Croatian-Bosnian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Gaelic$", "Scottish Gaelic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Greek$", "Modern Greek", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Punjabi$", "Eastern Panjabi", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Norman$", "Anglo-Norman", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Bikol Central$", "Coastal-Naga Bikol", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian (Eastern)$", "Eastern Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian (Western)$", "Western Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Manipuri (Meitei-Lon)$", "Manipuri", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Sumi)$", "Sumi Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Tangkhul)$", "North-Central Tangkhul Naga", str(x)) for x in df['language']] # A hypoglot
df['language'] = [re.sub(r"^Naga (Rengma)$", "Northern Rengma Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Lotha)$", "Lotha Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Konyak)$", "Konyak Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Chakhesang-Chokri)$", "Chokri Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Ao)$", "Ao Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Naga (Angami)$", "Angami Naga", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Kyrgyz$", "Kirghiz", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Hebrew$", "Modern Hebrew", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^North Sami$", "North Saami", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Greenlandic$", "Kalaallisut", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^West Frisian$", "Western Frisian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Javanese$", "Kawi", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old Church Slavonic$", "Church Slavic", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^North Frisian$", "Northern Frisian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Luxembourgish$", "Luxemburgish", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Armenian$", "Eastern Armenian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Old East Slavic$", "Old Russian", str(x)) for x in df['language']]
df['language'] = [re.sub(r"^Ossetian$", "Modern Ossetic", str(x)) for x in df['language']]

df_input = df

# load in datasets
languages=pd.read_csv(path_in_wiktionary + 'languages/languages.csv', header =[0], delimiter=',', encoding="utf-8", index_col=[0])

# merge input and languages
df = pd.merge(df_input, languages, on=['language'])
print("Merged:", df.shape)

#drop duplicates
df.drop_duplicates(subset=['language', 'term'], keep='first', inplace=True, ignore_index=True)
print("Dropping duplicates:", df.shape)

multilingual = df
multilingual

#check missing ones
temp = pd.merge(df_input, multilingual, how='outer', suffixes=('','_y'), indicator=True)
missing = temp[temp['_merge']=='left_only'][df_input.columns]
print("The following terms and languages have failed to load:")
print(missing)

# df = df.dropna() #OPERATIVE ONLY
# df = df.fillna('x')

#sort by categories, cinnamon ######## AUTOMATE ########
df['group'] = pd.Categorical(df['group'], ["canela", "kinnamon", "korica", "qirfa", "darchin", "gui", "other"]) # add categorical order here
df.sort_values("group", inplace = True) # sort according to the categories

# #sort by categories, pepper ######## AUTOMATE ########
# df['group'] = pd.Categorical(df['group'], ["pippali", "pigment", "marica", "hujiao", "other"]) # add categorical order here
# df.sort_values("group", inplace = True) # sort according to the categories

# create text for annotation label
df['text'] = df['term'] + '<br>' + df['transliteration'].astype(str) + '<br>Language: ' + df['language'] + '<br>Family: ' + df['family']
df['text'] = [re.sub(r"<br>nan<br>", "<br>", str(x)) for x in df['text']]

df['term'] = [re.sub(r"\u200e", "", str(x)) for x in df['term']] #removes right to left mark
df['term'] = [re.sub(r" *$", "", str(x)) for x in df['term']] #!

# reindex?

# save
df.to_csv(path_in_wiktionary + "multilingual/" + key +'.csv')
df

Before skipping:  (162, 6)
After skipping:  (148, 6)
Merged: (155, 16)
Dropping duplicates: (148, 16)
The following terms and languages have failed to load:
Empty DataFrame
Columns: [skip, language, term, transliteration, item, group]
Index: []


,skip,language,term,transliteration,item,group,iso,glcode,level,branch,family,macroarea,country,lat,lon,from,text
0,NaN,Afrikaans,kaneel,NaN,kaneel,canela,afr,afri1274,language,Germanic,Indo-European,Africa,ZA,-31.00000,22.00000,wals1,kaneel<br>Language: Afrikaans<br>Family: Indo-...
36,NaN,Estonian,kaneel,NaN,kaneel,canela,est,esto1258,language,Finnic,Uralic,Eurasia,EE,59.00000,26.00000,wals1,kaneel<br>Language: Estonian<br>Family: Uralic
37,NaN,Faroese,kanel,NaN,kanel,canela,fao,faro1244,language,Germanic,Indo-European,Eurasia,DK,62.00000,-7.00000,wals1,kanel<br>Language: Faroese<br>Family: Indo-Eur...
38,NaN,Finnish,kaneli,NaN,kaneli,canela,fin,finn1318,language,Finnic,Uralic,Eurasia,FI,62.00000,25.00000,wals1,kaneli<br>Language: Finnish<br>Family: Uralic
39,NaN,French,cannelle,NaN,cannelle,canela,fra,stan1290,language,Romance,Indo-European,Eurasia,CH FR,48.00000,2.00000,wals1,cannelle<br>Language: French<br>Family: Indo-E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,NaN,Sumi Naga,"losani, akusa",NaN,"losani, akusa",other,nsm,sumi1235,language,NaN,Sino-Tibetan,Eurasia,IN,25.99960,94.42350,glot,"losani, akusa<br>Language: Sumi Naga<br>Family..."
12,NaN,Brunei Malay,kayu manis,NaN,kayu manis,other,NaN,brun1243,dialect,NaN,Austronesian,Papunesia,NaN,4.53000,114.72000,glot,kayu manis<br>Language: Brunei Malay<br>Family...
146,NaN,Yoruba,eso igi gbigbẹ oloorun,NaN,eso igi gbigbẹ oloorun,other,yor,yoru1245,language,NaN,Atlantic-Congo,Africa,BJ NG,7.15345,3.67225,glot,eso igi gbigbẹ oloorun<br>Language: Yoruba<br>...
71,NaN,Lao,ອົບເຊຍ,obsey,obsey,other,lao,laoo1244,language,Kam-Tai,Tai-Kadai,Eurasia,LA TH,18.00000,103.00000,wals1,ອົບເຊຍ<br>obsey<br>Language: Lao<br>Family: Ta...
